## Datlinq data assessment
The first step is to import the the packages I'll be using. I'll start with Pandas for importing the data and initial cleaning of the data. Various parts of Scikit-Learn will become important in the next steps of processing the data.

In [1]:
import pandas as pd
from src.functions import import_json_to_df

### Data preparation
The first step is to import the data into Pandas to prepare it for processing. The function `import_json_to_df` uses `json_normalize` to flatten nested dicts inside the JSON into new columns. An unfortunate side-effect is that the columns are stored as object types when they should be numeric.

In [2]:
facebook_path = "data-sample/facebook-rotterdam-20170131.json"
factual_path = "data-sample/factual-rotterdam-20170207.csv"
google_path = "data-sample/google-rotterdam-20170207.json"

df_facebook = import_json_to_df(facebook_path)
df_factual = pd.read_csv(factual_path)
df_google = import_json_to_df(google_path)

fb_numeric_columns = df_facebook.columns.str.contains('numberLong')
df_facebook[df_facebook.columns[fb_numeric_columns]] = \
    df_facebook[df_facebook.columns[fb_numeric_columns]].apply(pd.to_numeric)

In [3]:
df_facebook.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14516 entries, 0 to 14515
Data columns (total 136 columns):
__timestamp_$numberLong                         int64
__timestamp_basic_$numberLong                   float64
__timestamp_detail_$numberLong                  float64
__timestamp_insights_$numberLong                float64
__timestamp_photos_$numberLong                  float64
about                                           object
affiliation                                     object
artists_we_like                                 object
attire                                          object
awards                                          object
band_interests                                  object
band_members                                    object
best_page_id                                    object
best_page_name                                  object
bio                                             object
birthday                                        object
booking